<br>

# Machine Learning algorithm
---
---


## Importing
---

In [17]:
import nltk
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

## Defining functions
---


In [3]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


def preprocess_word(word):
    # Remove punctuations at the beginning or end
    word = word.strip(string.punctuation)
    word = word.lower()  # Convert to lowercase
    if word.endswith("s"):
        word = word[:-1]  # Remove the trailing 's'
    return word


def most_common_word_in_list(input_text, word_list):
    words = input_text.lower().split()  # Convert to lowercase before splitting
    filtered_words = [
        preprocess_word(word) for word in words if preprocess_word(word) in word_list
    ]
    word_count = Counter(filtered_words)
    if word_count:
        return word_count.most_common(1)[0]
    else:
        return None

## Extracting and spliting pdf strings
---

In [18]:
pdf_words = extract_text_from_pdf("./test.pdf")
pdf_words = pdf_words.split()

<class 'list'>


## Creating categories and training data
---

In [4]:
a  # Define your categories and their corresponding data paths
categories = ["category1", "category2", "category3"]
data_paths = {
    "SO": [
        "/home/eduardotc/Zotero/storage/T8A7CK3F/Kuk et al. - 2023 - Singlet-oxygen-driven photocatalytic degradation o.pdf",
        "/home/eduardotc/Zotero/storage/MABMTDMW/Linger et al. - 2023 - Evaluation of relative efficiency of PDT photosens.pdf",
        "/home/eduardotc/Zotero/storage/KJ3YHB9R/Krajczewski et al. - 2019 - Role of various nanoparticles in photodynamic ther.pdf",
    ],
    "X-ray": [
        "/home/eduardotc/Zotero/storage/VGS9VKPU/Shaheen et al. - 2018 - In Vitro Cytotoxicity and Morphological Assessment.pdf",
        "/home/eduardotc/Zotero/storage/H7ZYSJCH/Clement et al. - 2016 - X-ray induced singlet oxygen generation by nanopar.pdf",
        "/home/eduardotc/Zotero/storage/P37C94Y4/Bulin et al. - 2013 - X-ray-Induced Singlet Oxygen Activation with Nanos.pdf",
    ],
    "h2o2": [
        "/home/eduardotc/Zotero/storage/LE68YYAB/Yang et al. - 2023 - Packing-induced selectivity switching in molecular.pdf",
        "/home/eduardotc/Zotero/storage/6JUNK2BN/Xu et al. - 2023 - Boosting alkaline photocatalytic H2O2 generation b.pdf",
    ],
}

# Create a list of (text, category) tuples for training
training_data = []
for category, files in data_paths.items():
    for file in files:
        text = extract_text_from_pdf(file)
        training_data.append((text, category))


nltk.download("punkt")

text_clf = make_pipeline(TfidfVectorizer(), MultinomialNB())

texts = [data[0] for data in training_data]
categories = [data[1] for data in training_data]

X_train, X_test, y_train, y_test = train_test_split(texts, categories, test_size=0.2)

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)

## Classifying a PDF
---

In [6]:
def classify_pdf(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    predicted_category = text_clf.predict([text])[0]
    return predicted_category

In [8]:
tst2 = classify_pdf(
    "/home/eduardotc/Zotero/storage/C84JU3XU/Chayanun et al. - 2020 - Direct Three-Dimensional Imaging of an X-ray Nanof.pdf"
)
print(tst2)

SO


<br>

# Most common words classifications
---
---

## Imports and definitions
---

In [10]:
import string
from collections import Counter

import nltk
import pandas as pd
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline


def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text


def is_numeric(word):
    try:
        float(word)
        return True
    except ValueError:
        return False


def preprocess_word(word):
    # Remove punctuations at the beginning or end
    word = word.strip(string.punctuation)
    word = word.lower()  # Convert to lowercase
    if word.endswith("s"):
        word = word[:-1]  # Remove the trailing 's'
    return word


def merge_rows(group):
    merged_value = sum(group["numbers"].astype(float))
    return pd.DataFrame({"word": [group["word"].iloc[0]], "numbers": [merged_value]})


def most_common_words(input_string, n, excluded_words):
    words = input_string.split()
    filtered_words = [
        word for word in words if word not in excluded_words and not is_numeric(word)
    ]
    word_count = Counter(filtered_words)
    return word_count.most_common(n)

## User defined variables
---

- pdf_path = String, pdf file path.

- higher_ns = Integer, Number of total most common words to be separated.

- excluded_words = dict, pdf words to be excluded from analysis

    - List of non importatn strings, to be filtered from the file results, mainly conjunctions, determiners, punctuation, personal names and initials, etc ...

    - *Numbers are filtered from the main strings list by the most_common_words function, not being necessary to add them to the bellow function.*


In [71]:
import pandas as pd
import numpy as np

pdf_path = "/home/eduardotc/Zotero/storage/ILZX7FWI/Farhana et al. - 2023 - Gold Nanoparticles Inhibit PMA-Induced MMP-9 Expre.pdf"
sort_categories = [
    "singlet",
    "cell",
    "cancer",
    "nanoparticle",
    "MOF",
    "oxygen",
    "luminescence",
    "mechanochemistry",
    "catalysis",
]
sort_subcategories = [
    "breast",
    "lung",
    "liver",
    "kidney",
    "Au",
    "Ag",
    "Nb",
    "Pd",
    "fluorescense",
    "upconversion",
    "nir",
    "laser",
    "light",
    "mass",
    "HPLC",
]

excluded_words = {
    "the",
    "of",
    "in",
    "and",
    "to",
    ".",
    "a",
    "is",
    "by",
    "this",
    "for",
    "with",
    "No",
    "be",
    "as",
    "The",
    "can",
    "at",
    "that",
    "are",
    "it",
    "A.",
    "B.",
    "C.",
    "D.",
    "E.",
    "F.",
    "G.",
    "H.",
    "I.",
    "J.",
    "K.",
    "L.",
    "M.",
    "N.",
    "O.",
    "P.",
    "Q.",
    "R.",
    "S.",
    "T.",
    "U.",
    "V.",
    "X.",
    "Y.",
    "Z.",
    "on",
    "or",
    "e",
    "O",
    "et",
    "This",
    "has",
    "In",
    "di",
    "very",
    "al.",
    "been",
    "Yes",
    "have",
    "which",
    "from",
    "will",
    "an",
    ",",
    "(",
    ")",
    "one",
    "other",
    "A",
    "but",
    "Section",
    "more",
    "most",
    "these",
    "It",
    "was",
    "not",
    "its",
    "some",
    "al.,",
    "As",
    "κ",
    "Figure",
    "These",
    "signiﬁcantly",
    "used",
    "were",
    "compared",
    "also",
    "(Figure",
    "CrossRef",
}

sort_df = {"Categories": sort_categories}
sort_df = pd.DataFrame(sort_df, index=idxcat)
print("\n")
sortsub_df = {"Subcategories": sort_subcategories}
sortsub_df = pd.DataFrame(sortsub_df, index=idxsubcat)
merged_df = pd.merge(sort_df, sortsub_df, how='outer', left_index=True, right_index=True)
print(merged_df)
n = 100



          Categories Subcategories
0            singlet        breast
1               cell          lung
2             cancer         liver
3       nanoparticle        kidney
4                MOF            Au
5             oxygen            Ag
6       luminescence            Nb
7   mechanochemistry            Pd
8          catalysis  fluorescense
9                NaN  upconversion
10               NaN           nir
11               NaN         laser
12               NaN         light
13               NaN          mass
14               NaN          HPLC


## Strings separation and counting
---


In [101]:
import pandas as pd

input_string = extract_text_from_pdf(pdf_path)
word_list = []
count_list = []
catls = []
numsum = []
common_words = most_common_words(input_string, n, excluded_words)
for word, count in common_words:
    word_list.append(preprocess_word(word))
    count_list.append(float(count))
df = {"word": word_list, "numbers": count_list}
df = pd.DataFrame(df)

grouped = df.groupby("word")
result_df = grouped.apply(merge_rows).reset_index(drop=True)
categories_count = []
ts = 0

for b in merged_df["Categories"]:
    ts = df.loc[df["word"] == b]
    ts = pd.DataFrame(ts)
    tts = sum(ts["numbers"])
    numsum.append(tts)
    catls.append(b)
dffin = {"words": catls, "numbers": numsum}
dffin = pd.DataFrame(dffin)
dffin = dffin.style.set_caption("Categories")

catls = []
numsum = []

for c in merged_df["Subcategories"]:
    ts = df.loc[df["word"] == c]
    ts = pd.DataFrame(ts)
    tts = sum(ts["numbers"])
    numsum.append(tts)
    catls.append(c)
    
dffin2 = {"words": catls, "numbers": numsum}
dffin2 = pd.DataFrame(dffin2)
dffin2 = dffin2.style.set_caption("Subcategories")
dffin
dffin2

,words,numbers
0,breast,70.000000
1,lung,0.000000
2,liver,0.000000
3,kidney,0.000000
4,Au,0.000000
5,Ag,0.000000
6,Nb,0.000000
7,Pd,0.000000
8,fluorescense,0.000000
9,upconversion,0.000000
